In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from itertools import product

# Type I : Generate all subset

# Feature ==> {f1, f2, f3}
## 1 --> {f1},{f2},{f3},{f1,f2},{f1,f3},{f2,f3},{F1,f2,f3},{}
## 2 --> {True,False,Flase},{False,True,False},..,{True,True,True},{False,Flase,False}

In [2]:
list(product({True, False}, repeat=3)) #เวลาใช้งานจริงไม่จำเป็นต้องแปลงเป็นลิสต์

[(False, False, False),
 (False, False, True),
 (False, True, False),
 (False, True, True),
 (True, False, False),
 (True, False, True),
 (True, True, False),
 (True, True, True)]

In [3]:
# Generate data
X, y = make_classification(n_samples= 1000, n_features= 5, n_informative=2,n_redundant=3, random_state=1)

In [4]:
X[0:6], y[0:6]

(array([[ 1.68999711,  0.31749358,  1.35806663, -1.52847073,  0.79430654],
        [ 0.71057852,  2.01526984, -1.89998791,  2.45923308,  3.2054298 ],
        [ 0.90269045,  0.60437891,  0.15445654, -0.09970463,  1.08773265],
        [ 0.14152804,  1.47267691, -1.78516076,  2.25571312,  2.27314571],
        [-0.07997162,  1.17843482, -1.63142209,  2.03961059,  1.78354504],
        [ 1.79133046,  0.12496373,  1.71731058, -1.96886321,  0.51909773]]),
 array([1, 1, 0, 1, 1, 1]))

In [5]:
best_score = 0
best_feature_set = []


for subset in product([True, False], repeat=5):
    # convert subset  => feature set แปลงเลือกไม่เลือกเป็ฯ feature
    # (True, False, True) ==> {f1, f3}
    index_col = [i for i, x in enumerate(subset) if x]
    #print(subset)
    #print(index_col) # index ของ feature ที่เราเลือก 
    #print('------------')
    if len(index_col) == 0:
        continue #ก็คือถ้าได้เป็นเซ็ตว่างจะไม่เอา ให้ข้ามไปต่อได้เลย
        
    selected_X = X[:, index_col] 
    
    ## Training 
    model_dt = DecisionTreeClassifier()
    score = cross_val_score(model_dt, selected_X, y, scoring = 'accuracy')
    overall_score = np.mean(score)
    
    print(index_col, '\t: overall_score = ', overall_score)
    
    if overall_score > best_score:
        best_score = overall_score
        best_feature_set = index_col
    
    

[0, 1, 2, 3, 4] 	: overall_score =  0.8300000000000001
[0, 1, 2, 3] 	: overall_score =  0.827
[0, 1, 2, 4] 	: overall_score =  0.8310000000000001
[0, 1, 2] 	: overall_score =  0.828
[0, 1, 3, 4] 	: overall_score =  0.8329999999999999
[0, 1, 3] 	: overall_score =  0.8230000000000001
[0, 1, 4] 	: overall_score =  0.82
[0, 1] 	: overall_score =  0.8240000000000001
[0, 2, 3, 4] 	: overall_score =  0.8160000000000001
[0, 2, 3] 	: overall_score =  0.812
[0, 2, 4] 	: overall_score =  0.827
[0, 2] 	: overall_score =  0.8160000000000001
[0, 3, 4] 	: overall_score =  0.8160000000000001
[0, 3] 	: overall_score =  0.8140000000000001
[0, 4] 	: overall_score =  0.808
[0] 	: overall_score =  0.655
[1, 2, 3, 4] 	: overall_score =  0.8220000000000001
[1, 2, 3] 	: overall_score =  0.8219999999999998
[1, 2, 4] 	: overall_score =  0.826
[1, 2] 	: overall_score =  0.8290000000000001
[1, 3, 4] 	: overall_score =  0.8210000000000001
[1, 3] 	: overall_score =  0.826
[1, 4] 	: overall_score =  0.798
[1] 	: ove

In [6]:
best_feature_set, best_score

([0, 1, 3, 4], 0.8329999999999999)

In [7]:
#model_dt = DecisionTreeClassifier()
#score = cross_val_score(model_dt, selected_X, y, scoring = 'accuracy')
#np.mean(score) #แบ่ง data เป็น partๆ เอาไป train test  = cross validation


# Type II : There are many feature

In [8]:
# Generate data
X, y = make_classification(n_samples= 10000, n_features= 500, n_informative=2,n_redundant=3, random_state=1)

In [9]:
def objective(X, y, sol):
    #convert sol --> selected feature columns เอาเฉพาะคอลัมที่เลือก
    index_col = [i for i, x in enumerate(sol) if x]
    
    #Empty set 
    if len(index_col) == 0:
        return 0, []
    
    selected_X = X[:, index_col] 
    
    ## Training 
    model_dt = DecisionTreeClassifier()
    score = cross_val_score(model_dt, selected_X, y, scoring = 'accuracy')
    overall_score = np.mean(score)    
    
    return overall_score, index_col

def modify_sol(sol):
    modified_sol = sol.copy()
    
    for i in range(len(modified_sol)):
        if np.random.rand() > 0.5:
            modified_sol[i] = not modified_sol[i]
            
    return modified_sol
        
        

def hillclimbing(X, y, objective, n_iter): 
    #random solution 
    initial_sol = np.random.choice([True, False], size = X.shape[1])
    
    #evaluate accuracy for the initial_sol
    best_acc, best_index_col = objective(X, y, initial_sol) 
    
    for i in range(n_iter):
        cadidate_sol = modify_sol(initial_sol)
        
        acc, index_col = objective(X, y, cadidate_sol) 
        print(i, 'best_acc = ', best_acc)
        
        if acc > best_acc:
            best_acc = acc
            best_index_col = index_col
        
    return best_acc, best_index_col
        

In [10]:
#np.random.choice([True, False], size = X.shape[1])

In [11]:
hillclimbing(X, y, objective, 10)

0 best_acc =  0.5109
1 best_acc =  0.9248000000000001
2 best_acc =  0.9248000000000001
3 best_acc =  0.9248000000000001
4 best_acc =  0.9248000000000001
5 best_acc =  0.9248000000000001
6 best_acc =  0.9248000000000001
7 best_acc =  0.9248000000000001
8 best_acc =  0.9248000000000001
9 best_acc =  0.9248000000000001


(0.9248000000000001,
 [1,
  3,
  6,
  7,
  9,
  10,
  12,
  13,
  14,
  20,
  21,
  25,
  28,
  30,
  31,
  32,
  33,
  34,
  37,
  38,
  39,
  41,
  42,
  43,
  47,
  49,
  50,
  51,
  52,
  53,
  54,
  56,
  57,
  58,
  60,
  61,
  63,
  64,
  65,
  66,
  69,
  76,
  77,
  79,
  81,
  85,
  87,
  90,
  91,
  92,
  98,
  99,
  101,
  102,
  103,
  106,
  109,
  110,
  111,
  113,
  115,
  119,
  120,
  122,
  124,
  126,
  130,
  133,
  134,
  139,
  143,
  147,
  150,
  151,
  152,
  153,
  154,
  155,
  157,
  158,
  161,
  164,
  173,
  174,
  176,
  178,
  180,
  184,
  185,
  186,
  187,
  188,
  192,
  193,
  194,
  195,
  196,
  198,
  199,
  200,
  201,
  203,
  205,
  208,
  209,
  211,
  214,
  215,
  220,
  221,
  222,
  223,
  224,
  225,
  228,
  235,
  239,
  241,
  245,
  252,
  256,
  257,
  258,
  259,
  261,
  265,
  267,
  269,
  270,
  273,
  274,
  276,
  277,
  278,
  280,
  282,
  285,
  286,
  288,
  289,
  290,
  291,
  293,
  301,
  303,
  306,
  307,
  308,
